In [1]:
import nppar
import time
import numpy as np
import math
import datatable as dt
import time
import xarray
import os
import fileutils
import traceback
from datetime import datetime
import multiprocessing
import random
from functools import partial
from multiprocessing import Pool, Array, Process, Manager
from functools import partial
import matplotlib.pyplot as plt

In [2]:
def gridinfo():
    rdiff=2227
    minnorth=333237
    #maxwest=1160624
    minwest=1156167
    maxeast=2747676
    maxsouth=3504175
    gridwidth=((maxeast-minwest) % rdiff) + 1
    firstid = minwest-math.ceil((minwest-minnorth) / rdiff)*rdiff
    gridheight=math.ceil((maxsouth-firstid) / rdiff)
    return rdiff, firstid, gridwidth, gridheight

def walkmonthdays(sfolder):
    #sfolder = '/data2/ffp/datasets/daily/2015/08'
    exfeat = ["id", "firedate"]
    dayfiles=[]
    for dayf in fileutils.find_files(sfolder, '*_norm.csv', listtype="walk"):
        dayfiles+=[dayf]
        #print(fday)
        '''
        try:
            #fday = '/data2/ffp/datasets/daily/2021/08/20210804_norm.csv'
            creategrid_xs(fday, rdiff, firstid, gridwidth, gridheight)
        except:
            print("Fail to convert %s"%fday)
            traceback.print_exc()
        '''
    return dayfiles

def call_creategrid(fday):
    creategrid_xs(fday, rdiff, firstid, gridwidth, gridheight)

def printduration(start, mess=''):
    duration = time.time() - start
    start = time.time()
    print("Duration %s %.1f" % (mess,duration))
    return start

def fill_grid_from_dict_np(_id, ggrid, firstid, iddict, rdiff, field="id"):
    print(_id)
    try:
        row,col = get_grid_xy(_id, firstid, rdiff)
        print(row,col)
        print(ggrid)
        if field=="id":
            ggrid[row,col]=_id
        elif field=="array":
            ggrid[row,col]=iddict[_id]
    except:
        print('dict: %s'%iddict[_id])

def fill_grid_from_dict_np(_id, ggrid, firstid, iddict, rdiff, field="id"):
    print(_id)
    try:
        row,col = get_grid_xy(_id, firstid, rdiff)
        print(row,col)
        print(ggrid)
        if field=="id":
            ggrid[row,col]=_id
        elif field=="array":
            ggrid[row,col]=iddict[_id]
    except:
        print('dict: %s'%iddict[_id])


def get_grid_xy(firstid, rdiff, _id,):
    row =int((_id-firstid)/rdiff)
    col = int(_id-firstid-rdiff*row)
    return row,col

def creategrid_np(fday, exfeat):
    rdiff, firstid, gridwidth, gridheight = gridinfo()
    start = time.time()
    #daydict, featcolumns = loaddaydict(fday, exfeat,100)
    #featn=len(featcolumns)
    start = printduration(start, "of loading day:")
    ggrid_id = np.zeros((gridwidth, gridheight))
    #ggrid_patchpool = np.zeros((gridheight, gridwidth, featn))
    #idsnp = np.array(list(daydict.keys()))
    fill_grid_from_dict_v=np.vectorize(fill_grid_from_dict_np, excluded=["ggrid", "firstid", "iddict", "rdiff", "field"] )
    print(type(ggrid_id))
    fill_grid_from_dict_v(idsnp, ggrid_id, firstid, daydict, rdiff)
    start = printduration(start, "of filling grid ids:")
    #fill_grid_from_dict(ggrid_patchpool, firstid, daydict, rdiff, "array")
    #start = printduration(start, "of filling grid features:")
    return ggrid_id, ggrid_patchpool


def creategrid_xa(rdiff, firstid, gridwidth, gridheight, fday):
    #fday = '/data2/ffp/datasets/daily/2021/08/20210803_norm.csv'
    dt_df = dt.fread(fday)
    npday = dt_df[:, dt_df.names.index('id'):].to_numpy(dt.float32)

    #start = time.time()
    id2xy, grid = nppar.fillcube(7, npday, firstid, rdiff, gridwidth, gridheight)
    #end = time.time()
    #print(end - start)

    xaday = xarray.DataArray(data=grid, dims=["x", "y", "feature"],
                             coords=dict(x=range(gridwidth), y=range(gridheight), feature=range(len(dt_df.names) - 1)))

    orig_path=os.path.dirname(fday)
    fname = os.path.basename(fday)
    xaday.to_dataset().to_netcdf(os.path.join(orig_path,"%s_grid.nc"%(fname[0:8])))

def creategrid_xs(rdiff, firstid, gridwidth, gridheight, dayfile, cpus):
    #fday = '/data2/ffp/datasets/daily/2021/08/20210803_norm.csv'
    #print("processing day %s" % dayfile)
    try:
        orig_path = os.path.dirname(dayfile)
        fname = os.path.basename(dayfile)
        daygrid="%s_grid.nc" % (fname[0:8])
        #if os.path.isfile(os.path.join(orig_path, daygrid)): return
        dt_df = dt.fread(dayfile)
        firstfeat=dt_df.names.index('id')
        npday = dt_df[:, firstfeat:].to_numpy(dt.float32)

        #start = time.time()
        id2xy, grid = nppar.fillcube(cpus, npday, firstid, rdiff, gridwidth, gridheight)
        #end = time.time()
        #print(end - start)

        vardict = {}
        for i in range(firstfeat, len(dt_df.names)):
            varname = dt_df.names[i]
            if dt_df.names[i] == 'x' or dt_df.names[i] == 'y':
                varname = '%spos' % varname
            vardict[varname] = (["x", "y", "time"], np.expand_dims(grid[:, :, i-firstfeat], axis=2))

        t = datetime.strptime(os.path.basename(dayfile)[0:8], '%Y%m%d')
        xsday = xarray.Dataset(data_vars=vardict, coords=dict(x=range(gridwidth), y=range(gridheight), time=[t]))
        xsday.to_netcdf(os.path.join(orig_path, daygrid))
        #print("Successfull convertion %s" % dayfile)
    except:
        print("Fail to convert %s" % dayfile)
        traceback.print_exc()
        with open("/data2/ffp/datasets/daily/failedgrids.log", "a") as f:
            f.write(dayfile)

def creategrid_xs_small(rdiff, firstid, gridwidth, gridheight, dayfile, pcpus, ccpus):
    # fday = '/data2/ffp/datasets/daily/2021/08/20210803_norm.csv'
    #print("processing day %s" % dayfile)
    try:
        orig_path = os.path.dirname(dayfile)
        fname = os.path.basename(dayfile)
        daygrid = "%s_grid.nc" % (fname[0:8])
        #if os.path.isfile(os.path.join(orig_path, daygrid)): return
        dt_df = dt.fread(dayfile, nthreads=1)#pcpus)
        firstfeat = dt_df.names.index('id')
        #npday = dt_df[:, firstfeat:].to_numpy(dt.float32)

        dynamic_feat=['id', 'max_temp', 'min_temp', 'mean_temp', 'res_max',
         'dom_vel', 'rain_7days', #'dem', 'slope', 'curvature', 'aspect',
         'ndvi_new', 'evi', 'lst_day', 'lst_night', 'max_dew_temp',
         'mean_dew_temp', 'min_dew_temp', 'fire', 'dir_max_1', 'dir_max_2',
         'dir_max_3', 'dir_max_4', 'dir_max_5', 'dir_max_6', 'dir_max_7',
         'dir_max_8', 'dom_dir_1', 'dom_dir_2', 'dom_dir_3', 'dom_dir_4',
         'dom_dir_5', 'dom_dir_6', 'dom_dir_7', 'dom_dir_8', #'corine_111',
        # 'corine_112', 'corine_121', 'corine_122', 'corine_123', 'corine_124',
        # 'corine_131', 'corine_132', 'corine_133', 'corine_141', 'corine_142',
        # 'corine_211', 'corine_212', 'corine_213', 'corine_221', 'corine_222',
        # 'corine_223', 'corine_231', 'corine_241', 'corine_242', 'corine_243',
        # 'corine_244', 'corine_311', 'corine_312', 'corine_313', 'corine_321',
        # 'corine_322', 'corine_323', 'corine_324', 'corine_331', 'corine_332',
        # 'corine_333', 'corine_334', 'corine_411', 'corine_412', 'corine_421',
        # 'corine_422', 'corine_511', 'corine_512', 'corine_521', 'wkd_0',
        # 'wkd_1', 'wkd_2', 'wkd_3', 'wkd_4', 'wkd_5', 'wkd_6', 'month_7',
        # 'month_4', 'month_5', 'month_6', 'month_8', 'month_9',
        'frequency','f81',]# 'xpos', 'ypos']

        dyn_df = dt_df[:, dynamic_feat]
        npday = dyn_df.to_numpy(dt.float32)

        # start = time.time()
        id2xy, grid = nppar.fillcube(ccpus, npday, firstid, rdiff, gridwidth, gridheight)
        # end = time.time()
        # print(end - start)

        vardict = {}
        for i in range(0, len(dyn_df.names)):
            varname = dyn_df.names[i]
            if dyn_df.names[i] == 'x' or dyn_df.names[i] == 'y':
                varname = '%spos' % varname
            vardict[varname] = (["x", "y", "time"], np.expand_dims(grid[:, :, i], axis=2))

        t = datetime.strptime(os.path.basename(dayfile)[0:8], '%Y%m%d')
        xsday = xarray.Dataset(data_vars=vardict, coords=dict(x=range(gridwidth), y=range(gridheight), time=[t]))
        xsday.to_netcdf(os.path.join(orig_path, daygrid))
        #print("Successfull convertion %s" % dayfile)
    except:
        print("Fail to convert %s" % dayfile)
        traceback.print_exc()
        with open("/data2/ffp/datasets/daily/failedgrids.log", "a") as f:
            f.write(dayfile)


def create_xs_files(creategrid, days, pcpus, ccpus):
    procs=[]
    dayscompleted=[]
    for cpu in range(pcpus):
        d=days.pop()
        dayscompleted+=[d]
        procs += [Process(target=creategrid, args=(d,pcpus,ccpus))]
        procs[cpu].start()
    while True:
        time.sleep(1)
        for p in procs:
            if not p.is_alive():
                procs.remove(p)
        if len(procs)<pcpus:
            if len(days)==0: break
            d = days.pop()
            dayscompleted += [d]
            procs += [Process(target=creategrid, args=(d,pcpus,ccpus))]
            procs[-1].start()

def assignrow(ggrid, tabrow):
    row, col = get_xy(tabrow[0])
    ggrid[row, col, :]=tabrow[:]

def assignrowshared(ggrid_sh, grid_shape, tabrows):
    for i in range(tabrows.shape[0]):
        tabrow=tabrows[i,:]
        try:
            row, col = get_xy(tabrow[0])
            idx = row*grid_shape[1]*grid_shape[2]+col*grid_shape[2]
            ggrid_sh[idx:idx+grid_shape[2]]=tabrow[:]
        except:
            "Error row: %s\n"%i+traceback.print_exc()

In [3]:
#initialize
rdiff, firstid, gridwidth, gridheight = gridinfo()
creategrid = partial(creategrid_xs_small, rdiff, firstid, gridwidth, gridheight)
get_xy=partial(get_grid_xy, firstid, rdiff)
dayfiles=walkmonthdays('/data2/ffp/datasets/daily/')
fday='/data2/ffp/datasets/daily/2021/08/20210823_norm.csv'
dt_df = dt.fread(fday)
#creategrid_xs(rdiff, firstid, gridwidth, gridheight, fday)
firstfeat=dt_df.names.index('id')
npday = dt_df[:, firstfeat:].to_numpy(dt.float32)
#npday = dt_df[:, 1:].to_numpy(dt.float32)
maxcpus=multiprocessing.cpu_count()
featn = len(dt_df[:, firstfeat:].names)
ggrid = np.zeros((gridwidth, gridheight, featn))
ggrid[:,:]=np.nan
assignr=partial(assignrow, ggrid)
#assignr(list(npday)[0])
print('max cpu count %s'%maxcpus)
print('array rows: %s'%npday.shape[0])

max cpu count 16
array rows: 372918


### Simple python run

In [5]:
nruns=1
totalrun=0
start = time.time()
for i in range(npday.shape[0]):
    assignr(npday[i])
end = time.time()
print('time: %.2f sec'%(end-start))

time: 3.80 sec


### Multiprocessing with map

In [6]:
nruns=1
totalrun=0
npdayl=list(npday)
for cpus in range(10,11):
    for run in range(nruns):
        start = time.time()
        with Pool(cpus) as p:
            p.map(assignr, npdayl)
        end = time.time()
        totalrun+=(end - start)
    print('average time: %.2f sec, cpus: %s'%(totalrun/nruns, cpus))
npdayl=None

average time: 169.95 sec, cpus: 10


### multiprocessing with shared memory

In [11]:
nruns=1
totalrun=0
ncpus=10
for ncpus in range(1,8):
    start = time.time()
    procs=[]
    chunk = int(npday.shape[0] / ncpus)
    chunk_rem = npday.shape[0] % ncpus
    shDay = Array('f', gridwidth * gridheight * featn)
    for cpu in range(ncpus-1):
        procs += [Process(target=assignrowshared, args=(shDay, (gridwidth, gridheight, featn), npday[cpu*chunk:(cpu+1)*chunk]))]
        procs[cpu].start()
    procs += [Process(target=assignrowshared, args=(shDay, (gridwidth, gridheight, featn), npday[(ncpus-1)*chunk:]))]
    procs[-1].start()
    #while True:
    #    time.sleep(1)
    #    if all([not proc.is_alive() for proc in procs]):
    #        break
    for p in procs: p.join()
    shDay_np = np.frombuffer(shDay.get_obj(), dtype=np.float32).reshape((gridwidth, gridheight, featn))
    end = time.time()
    print('time: %.2f sec, cpu: %s'%((end-start),ncpus))

array rows: 372918
time: 7.71 sec, cpu: 1
time: 8.10 sec, cpu: 2
time: 8.40 sec, cpu: 3


Traceback (most recent call last):
Traceback (most recent call last):
  File "<ipython-input-3-ad447073c9b3>", line 225, in assignrowshared
    row, col = get_xy(tabrow[0])
  File "<ipython-input-3-ad447073c9b3>", line 225, in assignrowshared
    row, col = get_xy(tabrow[0])
  File "<ipython-input-3-ad447073c9b3>", line 67, in get_grid_xy
    row =int((_id-firstid)/rdiff)
  File "<ipython-input-3-ad447073c9b3>", line 67, in get_grid_xy
    row =int((_id-firstid)/rdiff)
KeyboardInterrupt
KeyboardInterrupt
Process Process-154:
Process Process-155:
Traceback (most recent call last):
Traceback (most recent call last):
  File "<ipython-input-3-ad447073c9b3>", line 225, in assignrowshared
    row, col = get_xy(tabrow[0])
  File "<ipython-input-3-ad447073c9b3>", line 225, in assignrowshared
    row, col = get_xy(tabrow[0])
  File "<ipython-input-3-ad447073c9b3>", line 67, in get_grid_xy
    row =int((_id-firstid)/rdiff)
  File "<ipython-input-3-ad447073c9b3>", line 67, in get_grid_xy
    row 

KeyboardInterrupt: 

KeyboardInterrupt
Process Process-156:
Traceback (most recent call last):
  File "<ipython-input-3-ad447073c9b3>", line 227, in assignrowshared
    ggrid_sh[idx:idx+grid_shape[2]]=tabrow[:]
  File "/data2/ffp/envs/ml_vm2/lib/python3.9/multiprocessing/sharedctypes.py", line 226, in __setitem__
    with self:
  File "/data2/ffp/envs/ml_vm2/lib/python3.9/multiprocessing/sharedctypes.py", line 193, in __enter__
    return self._lock.__enter__()
  File "/data2/ffp/envs/ml_vm2/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/data2/ffp/envs/ml_vm2/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/data2/ffp/envs/ml_vm2/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-ad447073c9b3>", line

### cython (with shared memory) cpu 

In [8]:
nruns=20
print('cpu changes, runs per experiment: %s'%nruns)
for nt in range(1,maxcpus+1):
    totalrun = 0
    for run in range(nruns):
        start = time.time()
        id2xy, grid = nppar.fillcube(nt, npday, firstid, rdiff, gridwidth, gridheight)
        end = time.time()
        totalrun+=(end - start)
    print('average time: %.2f sec, threads: %s'%(totalrun/nruns,nt))

cpu changes, runs per experiment: 20
average time: 0.62 sec, threads: 1
average time: 0.43 sec, threads: 2
average time: 0.36 sec, threads: 3
average time: 0.33 sec, threads: 4
average time: 0.30 sec, threads: 5
average time: 0.33 sec, threads: 6
average time: 0.32 sec, threads: 7
average time: 0.31 sec, threads: 8
average time: 0.27 sec, threads: 9
average time: 0.26 sec, threads: 10
average time: 0.27 sec, threads: 11
average time: 0.26 sec, threads: 12
average time: 0.26 sec, threads: 13
average time: 0.26 sec, threads: 14
average time: 0.25 sec, threads: 15
average time: 0.25 sec, threads: 16


### cython (with shared memory) chunks 

In [13]:
print('chunk changes')
nt = 10
print('rows / threads: %.0f threads: %s' % ((npday.shape[0] / nt),nt))
nruns=20
for cs in range(1,400000,10000):
    totalrun = 0
    for run in range(nruns):
        start = time.time()
        id2xy, grid = nppar.fillcube(nt, npday, firstid,  rdiff, gridwidth, gridheight, 'static', cs)
        end = time.time()
        totalrun+=(end - start)
    print('average time: %.2f chunk: %s'%(totalrun/nruns, cs))

chunk changes
rows / threads: 37292 threads: 10
time: 0.32 chunk: 1
time: 0.28 chunk: 10001
time: 0.28 chunk: 20001
time: 0.29 chunk: 30001
time: 0.26 chunk: 40001
time: 0.27 chunk: 50001
time: 0.28 chunk: 60001
time: 0.29 chunk: 70001
time: 0.31 chunk: 80001
time: 0.32 chunk: 90001
time: 0.33 chunk: 100001
time: 0.34 chunk: 110001
time: 0.36 chunk: 120001
time: 0.37 chunk: 130001
time: 0.38 chunk: 140001
time: 0.39 chunk: 150001
time: 0.40 chunk: 160001
time: 0.41 chunk: 170001
time: 0.43 chunk: 180001
time: 0.43 chunk: 190001
time: 0.44 chunk: 200001
time: 0.45 chunk: 210001
time: 0.46 chunk: 220001
time: 0.48 chunk: 230001
time: 0.49 chunk: 240001
time: 0.50 chunk: 250001
time: 0.51 chunk: 260001
time: 0.52 chunk: 270001
time: 0.53 chunk: 280001
time: 0.54 chunk: 290001
time: 0.54 chunk: 300001
time: 0.58 chunk: 310001
time: 0.58 chunk: 320001
time: 0.58 chunk: 330001
time: 0.58 chunk: 340001
time: 0.59 chunk: 350001
time: 0.61 chunk: 360001
time: 0.62 chunk: 370001
time: 0.63 chunk

### cython (with shared memory) OpenMP schedules

In [ ]:
nt = 14
nruns=30
print('optimum rows / threads: %.0f, threads: %s, runs per experiment: %s' % ((npday.shape[0] / nt),nt, nruns))
for i in range(3):
    id2xy, grid = nppar.fillcube(nt, npday, firstid,  rdiff, gridwidth, gridheight, None)
schedules = [None, 'static', 'dynamic', 'guided']
random.shuffle(schedules)
for schedule in schedules:
    totalrun = 0
    for run in range(nruns):
        start = time.time()
        id2xy, grid = nppar.fillcube(nt, npday, firstid,  rdiff, gridwidth, gridheight, schedule)
        end = time.time()
        totalrun+=(end - start)
    print('average time: %.2f sec, schedule: %s'%(totalrun/nruns, schedule))

#xaday=xarray.DataArray(data=grid, dims=["x", "y", "feature"],  coords=dict(x=range(gridwidth), y=range(gridheight), feature=range(len(dt_df.names)-1)))



In [ ]:
t = datetime.strptime(os.path.basename(fday)[0:8], '%Y%m%d')

vardict={}
for i in range(1,len(dt_df.names)):
    varname=dt_df.names[i]
    if dt_df.names[i]=='x' or dt_df.names[i]=='y':
        varname='%spos'%varname
    vardict[varname]=(["x", "y", "time"], np.expand_dims(grid[:, :, i - 1], axis=2))

xsday=xarray.Dataset(data_vars=vardict, coords=dict(x=range(gridwidth), y=range(gridheight), time=[t]))

### Table Conversion plus IO read/write (csv to netcdf)

In [4]:
nruns=1
totalrun=0
for ccpus in range(5,16,5):
    for pcpus in range(5, 11, 5):
        start = time.time()
        create_xs_files(creategrid, dayfiles[:10], pcpus, ccpus)
        end = time.time()
        print('time: %.2f sec, python threads %s, cython threads %s' % (end - start, pcpus, ccpus))

time: 19.37 sec, python threads 5, cython threads 5
time: 28.71 sec, python threads 10, cython threads 5
time: 38.52 sec, python threads 5, cython threads 10
time: 22.45 sec, python threads 10, cython threads 10
time: 23.47 sec, python threads 5, cython threads 15
time: 21.46 sec, python threads 10, cython threads 15
